# Age Info

In [1]:
import pandas as pd

path = 'raw/local_authority/ONS-population_2021-08-05.csv'

ltla_popu_df = pd.read_csv(path)


L = []

for areaCode, df in ltla_popu_df.groupby('areaCode'):

    d = {}
    age_items = [ '15_19', '20_24', '25_29', '30_34', '35_39', '40_44', '45_49', '50_54', '55_59',  '60_64']
    popu_15_64 = df[df['age'].isin(age_items)]['population'].sum()
    # print(popu_15_64)
    if pd.isnull(popu_15_64) or popu_15_64 == 0:
        # print(popu_15_64)
        continue
        
    old_age_items = ['65_69', '70_74', '75_79', '80_84', '85_89', '90+']
    popu_65_above = df[df['age'].isin(old_age_items)]['population'].sum()
    # print(popu_65_above)
    # print(popu_65_above / popu_15_64)
    if pd.isnull(popu_65_above) or popu_65_above == 0:
        # print(popu_65_above)
        continue
        
    young_age_items = ['00_04', '05_09', '10_14',]
    popu_14_below = df[df['age'].isin(young_age_items)]['population'].sum()
    # print(popu_14_below)
    # print(popu_14_below / popu_15_64)
    if pd.isnull(popu_14_below) or popu_14_below == 0:
        # print(popu_14_below)
        continue
        

    # population
    popu_all = df[df['category'] == 'ALL']['population'].values[0]

    # print(popu_15_64, popu_65_above, popu_14_below, popu_all)
    # print(sum([popu_15_64, popu_65_above, popu_14_below]), popu_all)
    
    old_dependency_ratio = popu_65_above / popu_15_64
   
        
    young_dependency_ratio = popu_14_below / popu_15_64
   
    
    d['areaCode'] = areaCode
    d['old_dependency_ratio'] = old_dependency_ratio
    d['young_dependency_ratio'] = young_dependency_ratio
    d['popu_all'] = popu_all
    
    
    # print(old_dependency_ratio)
    # print(young_dependency_ratio)
    # print(popu_all)
    L.append(d)
    
    
ltla_demo_df = pd.DataFrame(L)
# ltla_demo_df = ltla_demo_df.rename(columns = {'areaCode': 'Area Code'})
ltla_demo_df

,areaCode,old_dependency_ratio,young_dependency_ratio,popu_all
0,E06000001,0.313702,0.288634,93836
1,E06000002,0.258597,0.315276,141285
2,E06000003,0.378299,0.280907,137228
3,E06000004,0.299317,0.299981,197419
4,E06000005,0.333318,0.284279,107402
...,...,...,...,...
441,W06000021,0.429542,0.249872,95164
442,W06000022,0.267686,0.303509,156447
443,W06000023,0.480443,0.256286,133030
444,W06000024,0.299149,0.287282,60424


# Popu Density

In [2]:
path = 'raw/local_authority/population_profiles_for_local_authorities_in_england.xlsx'

median_age_density_df = pd.read_excel(path, sheet_name = 'Median age & Population Density', skiprows = 4)#[:, ~5]

cols = ['Region code', 'Region name', 'Area code', 'Area name ', 'Median age', 'People per sq. km']
ltla_median_age_density_df = median_age_density_df[cols].rename(columns = {'Area code': 'areaCode'})


path = 'raw/local_authority/population_profiles_for_local_authorities_in_england.xlsx'

ltla_deprivation_df = pd.read_excel(path, sheet_name = 'Deprivation', skiprows = 5).rename(columns = {'Area Code': 'areaCode', 
                                                                                                     'Average Score ': 'Average Score'})


path = 'raw/local_authority/ukpopestimatesmid2020on2021geography.xls'
cols = ['Code', 'Area (sq km)']

ltla_area_df = pd.read_excel(path, sheet_name = 'MYE 5', skiprows = 7)[cols].rename(columns = {'Code': 'areaCode'})



df = pd.merge(ltla_median_age_density_df[['areaCode', 'Median age', 'People per sq. km']], ltla_deprivation_df[['areaCode', 'Average Score']], on = 'areaCode')
ltla_popu_density_df = pd.merge(df, ltla_area_df, on = 'areaCode')
ltla_popu_density_df


,areaCode,Median age,People per sq. km,Average Score,Area (sq km)
0,E92000001,40.0,432,NaN,130310.4928
1,E12000001,41.8,311,28.0,8580.7740
2,E12000002,40.3,520,28.1,14108.0004
3,E12000003,40.1,357,26.0,15404.4614
4,E12000004,41.4,309,20.3,15623.5214
...,...,...,...,...,...
312,E07000083,44.5,229,12.1,414.4144
313,E07000187,47.0,156,16.6,739.4356
314,E07000188,46.9,218,21.3,564.2238
315,E07000246,47.9,131,19.1,1187.5948


# Gender Info

In [3]:
def calculate_average_age(x):
    total_popu = 0
    total_age = 0
    
    for i in range(0, 90):
        age_popu = x[str(i)]
        age_sum = i*age_popu
        total_popu = total_popu + age_popu
        total_age = total_age + age_sum
        
    # 90+
    age_popu = x['90+']
    age_sum = 90*x['90+']
    total_popu = total_popu + age_popu
    total_age = total_age + age_sum
    
    average_age = total_age / total_popu
    return average_age



path = 'raw/local_authority/ukpopestimatesmid2020on2021geography.xls'
ltla_popu_df = pd.read_excel(path, sheet_name = 'MYE2 - Persons', skiprows = 7)#[cols]
# ltla_popu_df


path = 'raw/local_authority/ukpopestimatesmid2020on2021geography.xls'
ltla_popu_male_df = pd.read_excel(path, sheet_name = 'MYE2 - Males', skiprows = 7)#[cols]
# ltla_popu_male_df


path = 'raw/local_authority/ukpopestimatesmid2020on2021geography.xls'
ltla_popu_female_df = pd.read_excel(path, sheet_name = 'MYE2 - Females', skiprows = 7)#[cols]
# ltla_popu_female_df

ltla_popu_df['avg_age'] = ltla_popu_df.apply(lambda x: calculate_average_age(x), axis = 1)
ltla_popu_male_df['male_avg_age'] = ltla_popu_male_df.apply(lambda x: calculate_average_age(x), axis = 1)
ltla_popu_female_df['female_avg_age'] = ltla_popu_female_df.apply(lambda x: calculate_average_age(x), axis = 1)

ltla_popu_male_df['male_rate'] = ltla_popu_male_df['All ages'] / ltla_popu_df['All ages']
ltla_popu_female_df['female_rate'] = ltla_popu_female_df['All ages'] / ltla_popu_df['All ages']
df1 = ltla_popu_male_df[['Code', 'male_avg_age']]
df2 = ltla_popu_female_df[['Code', 'female_avg_age', 'female_rate']]

ltla_gender_df = pd.merge(df1, df2, on = 'Code').rename(columns = {'Code': 'areaCode'})
ltla_gender_df

,areaCode,male_avg_age,female_avg_age,female_rate
0,K02000001,39.538166,41.430686,0.505887
1,K03000001,39.574782,41.466605,0.505850
2,K04000001,39.478681,41.351731,0.505250
3,E92000001,39.401787,41.266400,0.505168
4,E12000001,40.447852,42.468675,0.508443
...,...,...,...,...
415,N09000006,38.748118,40.254941,0.498462
416,N09000007,38.941935,41.369442,0.508802
417,N09000008,40.279611,42.194942,0.509025
418,N09000009,36.668395,38.109700,0.497486


# LTAL Covariates

In [6]:
df = pd.merge(ltla_demo_df, ltla_popu_density_df, on = 'areaCode')
ltal_covariates_df = pd.merge(df, ltla_gender_df, on = 'areaCode')
ltal_covariates_df.to_csv('data/ltal_covariates_df.csv', index = False)
ltal_covariates_df

,areaCode,old_dependency_ratio,young_dependency_ratio,popu_all,Median age,People per sq. km,Average Score,Area (sq km),male_avg_age,female_avg_age,female_rate
0,E06000001,0.313702,0.288634,93836,41.7,999,35.0,93.7125,40.157418,42.190274,0.509730
1,E06000002,0.258597,0.315276,141285,36.3,2616,40.5,53.8816,37.015974,39.302367,0.505057
2,E06000003,0.378299,0.280907,137228,45.2,560,29.8,245.0698,42.037639,44.115800,0.514633
3,E06000004,0.299317,0.299981,197419,40.7,963,25.8,204.9331,39.646713,41.726862,0.505058
4,E06000005,0.333318,0.284279,107402,43.2,541,25.7,197.4758,41.155788,42.778638,0.513445
...,...,...,...,...,...,...,...,...,...,...,...
310,E12000006,0.323239,0.296275,6269161,41.7,326,17.4,19115.5929,40.259629,42.157051,0.507200
311,E12000007,0.178577,0.284969,9002488,35.6,5701,21.8,1572.0309,35.938778,37.519285,0.498541
312,E12000008,0.315837,0.290755,9217265,41.7,481,15.5,19071.8553,40.139930,42.136564,0.507159
313,E12000009,0.367664,0.270264,5659143,44.1,236,18.2,23835.9902,41.717684,43.778184,0.507886
